<a href="https://colab.research.google.com/github/lechemrc/Datasets-to-ref/blob/master/Pitchfork%20Reviews/Pitchfork_Reviews_Data_Project_LS_Unit_2_merging_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [2]:
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install eli5
    !pip install pandas-profiling==2.*
    !pip install pdpbox
    !pip install shap

     |████████████████████████████████| 102kB 2.6MB/s 
     |████████████████████████████████| 112kB 2.8MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 614kB 46.7MB/s 
     |████████████████████████████████| 235kB 47.9MB/s 
     |████████████████████████████████| 307kB 50.5MB/s 
     |████████████████████████████████| 215kB 57.1MB/s 
     |████████████████████████████████| 51kB 25.1MB/s 
     |████████████████████████████████| 51kB 24.0MB/s 
     |████████████████████████████████| 737kB 51.4MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145035 sha256=508eaf8a368a6a73251528e515ea2a82cea702ad1077dce17de0f974e5f93233
  Stored in directory: /root/.cache/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=304db5a21aceb6a72ec2267f80ab21011bcad63705860e7ae5bc15b3f774cc17
  Stored 

### Important imports / format

In [0]:
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# libraries and math functions
import pandas as pd
import numpy as np
import pandas_profiling
from scipy.io import arff # for loading .arff file
from scipy.stats import randint, uniform
import random

# imports for pipeline and regression
import category_encoders as ce
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.utils.multiclass import unique_labels
from ipywidgets import interact, fixed
from xgboost import XGBRegressor, XGBClassifier
from pdpbox import pdp
from pdpbox.pdp import pdp_isolate, pdp_plot, pdp_interact, pdp_interact_plot
import eli5
from eli5.sklearn import PermutationImportance
import shap

# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Using TensorFlow backend.


### Importing Data

In [5]:

url_list = ['https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/artists.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/genres.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/labels.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/reviews.csv',
            'https://raw.githubusercontent.com/lechemrc/Datasets-to-ref/master/Pitchfork%20Reviews/years.csv']

artists = pd.read_csv(url_list[0])
genres = pd.read_csv(url_list[1])
labels = pd.read_csv(url_list[2])
reviews = pd.read_csv(url_list[3])
years = pd.read_csv(url_list[4])

artists.shape, genres.shape, labels.shape, reviews.shape, years.shape

((18831, 2), (22680, 2), (20190, 2), (18393, 13), (19108, 2))

In [12]:
years.head()

,reviewid,year
0,22703,1998.0
1,22721,2016.0
2,22659,2016.0
3,22661,2016.0
4,22725,2016.0


### Merging Dataframes

In [13]:
# Merging the dataframes
df = artists.merge(
    genres, how='left', on='reviewid').merge(
        labels, how='left', on='reviewid').merge(
            reviews, how='left', on='reviewid').merge(
                years, how='left', on='reviewid')
            
print(df.shape)
df.head()

(26902, 17)


,reviewid,artist_x,genre,label,title,artist_y,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,year
0,22703,massive attack,electronic,virgin,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,1998.0
1,22721,krallice,metal,hathenter,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017,2016.0
2,22659,uranium club,rock,static shock,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,2016.0
3,22659,uranium club,rock,fashionable idiots,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,2016.0
4,22661,kleenex,rock,kill rock stars,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,2016.0


### Export CSV file

In [0]:
# from google.colab import files
# df.to_csv('pitchfork_reviews.csv') 
# files.download('pitchfork_reviews.csv')